In [1]:
import numpy as np
import pandas as pd

import re
from itertools import chain
from collections import Counter

In [2]:
_PUNCTUATION = "!\"#$%&'()*+,-./:;<=>?@[\\]^_`{|}~"
_PUNC_TABLE = str.maketrans("", "", _PUNCTUATION)
_WORD_REGEX = re.compile(r"(?u)\b\w\w+\b")  # sklearn default

with open('dictionaries/ukrainian-stopwords.txt', encoding="utf-8") as f:
    _STOP_WORDS = f.read().split(sep='\n')
    
    
def strip_punctuation(line):
    """Remove punctuation from a string"""
    return line.translate(_PUNC_TABLE).strip()

# def strip_inflection(line):
#     return [_LEM_DICT.get(w, w) for w in line]

    
def remove_stop_words(words):
    """Remove stop words from a list of word strings"""
    return [w for w in words if w not in _STOP_WORDS]


def tokenize_words(line, lowercase=True, filter_stopwords=True):
    """
    Split a string into individual lower-case words, optionally removing
    punctuation and stop-words in the process
    """
    words = _WORD_REGEX.findall(line.lower() if lowercase else line)
    return remove_stop_words(words) if filter_stopwords else words

In [3]:
df = pd.read_csv("./data/pravda_november.csv")

tone_dict = pd.read_csv('dictionaries/tone-dict-uk.tsv', sep='\t', names=['word', 'tone'])
lemmatiz_dict = pd.read_csv('dictionaries/lemmatization-uk.txt', sep="\t", names = ['to', 'from'])

In [4]:
# lemmas
lemmatiz_dict.index = lemmatiz_dict['from']
lemmatiz_dict = lemmatiz_dict.drop('from', axis = 1).to_dict() 
lemmatiz_dict = lemmatiz_dict['to']

# tones
tone_dict.word = tone_dict.word.str.lower()
tone_dict.tone = (tone_dict.tone > 0).astype(int)
tone_dict.loc[tone_dict.tone == 0, 'tone'] = -1
tone_dict = dict(tone_dict.values)

In [5]:
df["lemmatized"] = (df["title"]
                     .apply(lambda s: strip_punctuation(s)).str.split()
                     .apply(lambda s: [lemmatiz_dict.get(w,w) for w in s])
                     .apply(lambda s: tokenize_words(" ".join(s)))
                    )

In [6]:
df.head()

,title,subtitle,date,link,lemmatized
0,"В СБУ розповіли, чого бракує в розслідуванні в...",У правоохоронців немає достатніх доказів для п...,2019/11/1 23:44,https://www.pravda.com.ua/news/2019/11/1/7230775/,"[сбу, розповісти, бракувати, розслідування, вб..."
1,"Економічні новини: закон про фінмоніторинг, ре...",Про законопроєкт фінмоніторингу. Верховна Рада...,2019/11/1 23:03,https://www.epravda.com.ua/news/2019/11/1/653247/,"[економічні, новина, закон, фінмоніторинг, рей..."
2,Через смерть підлітка відсторонили керівника п...,"Розслідуванням смерті 14-річного хлопця, знайд...",2019/11/1 22:41,https://www.pravda.com.ua/news/2019/11/1/7230773/,"[смерть, підліток, відсторонили, керівник, пол..."
3,З листопада у наземному транспорті можна плати...,Сплатити за проїзд за допомогою банківської ка...,2019/11/1 22:05,https://www.epravda.com.ua/news/2019/11/1/653254/,"[листопад, наземний, транспорт, платити, смарт..."
4,ГПУ почала розслідувати погрози Федини Зеленсь...,Генеральна прокуратура відкрила кримінальне пр...,2019/11/1 21:35,https://www.pravda.com.ua/news/2019/11/1/7230730/,"[гпу, почати, розслідувати, погроза, федини, з..."


In [7]:
df["tone_words"] = df["lemmatized"].apply(lambda s: [w for w in s if w in tone_dict])

In [8]:
df.head()

,title,subtitle,date,link,lemmatized,tone_words
0,"В СБУ розповіли, чого бракує в розслідуванні в...",У правоохоронців немає достатніх доказів для п...,2019/11/1 23:44,https://www.pravda.com.ua/news/2019/11/1/7230775/,"[сбу, розповісти, бракувати, розслідування, вб...","[бракувати, вбивство]"
1,"Економічні новини: закон про фінмоніторинг, ре...",Про законопроєкт фінмоніторингу. Верховна Рада...,2019/11/1 23:03,https://www.epravda.com.ua/news/2019/11/1/653247/,"[економічні, новина, закон, фінмоніторинг, рей...",[]
2,Через смерть підлітка відсторонили керівника п...,"Розслідуванням смерті 14-річного хлопця, знайд...",2019/11/1 22:41,https://www.pravda.com.ua/news/2019/11/1/7230773/,"[смерть, підліток, відсторонили, керівник, пол...",[смерть]
3,З листопада у наземному транспорті можна плати...,Сплатити за проїзд за допомогою банківської ка...,2019/11/1 22:05,https://www.epravda.com.ua/news/2019/11/1/653254/,"[листопад, наземний, транспорт, платити, смарт...",[]
4,ГПУ почала розслідувати погрози Федини Зеленсь...,Генеральна прокуратура відкрила кримінальне пр...,2019/11/1 21:35,https://www.pravda.com.ua/news/2019/11/1/7230730/,"[гпу, почати, розслідувати, погроза, федини, з...",[]


In [9]:
print(f'Number of rows with tone is {(df.tone_words.apply(len) > 0).value_counts()[True]}, '
      f'without tone is {df.tone_words.apply(len).value_counts()[0]}'
     )

Number of rows with tone is 871, without tone is 1319


In [10]:
df["negative_tone"] = df["tone_words"].apply(lambda s: any(tone_dict[w] == -1 for w in s))

In [11]:
round((sum(df.negative_tone) / len(df)) * 100, ndigits=1)

28.5

In [12]:
common_all = Counter(chain(*df["lemmatized"]))
print('Total words', sum(common_all.values()))
common_all.most_common(10)

Total words 14605


[('україна', 209),
 ('зеленський', 92),
 ('київ', 83),
 ('справа', 82),
 ('сша', 78),
 ('рф', 71),
 ('суд', 67),
 ('народ', 65),
 ('новий', 62),
 ('донбас', 59)]

In [13]:
df["tone_words_neg"] = df["tone_words"].apply(lambda s: [w for w in s if tone_dict[w] < 0])

In [14]:
neg_counter = Counter(chain(*df.tone_words_neg))
neg_counter.most_common(10)

[('затримати', 34),
 ('підозра', 24),
 ('загинути', 22),
 ('обшук', 18),
 ('вибух', 17),
 ('арешт', 16),
 ('вбивство', 15),
 ('вимагати', 15),
 ('порушення', 14),
 ('стріляти', 13)]